In [28]:
import numpy as np
import pandas as pd
import torch
import pickle

In [ ]:
def linear_anneal(t, T, start, final, percentage):
    ''' Linear annealing scheduler
    t: current timestep
    T: total timesteps
    start: initial value
    final: value after percentage*T steps
    percentage: percentage of T after which annealing finishes
    '''
    final_from_T = int(percentage * T)
    if t > final_from_T:
        return final
    else:
        return final + (start - final) * (final_from_T - t) / final_from_T

In [5]:
linear_anneal(17, 25, 1.0, 0.01, 0.8)

0.1585

In [6]:
import torch.nn as nn


def build_nn():
    input_size = 72
    output = 8
    layer_sizes = [input_size] + [128, 128, 128] + [output]

    assert len(layer_sizes) > 1
    layers = []
    for index in range(len(layer_sizes) - 1):
        linear = nn.Linear(layer_sizes[index], layer_sizes[index + 1])
        act = nn.ReLU() if index < len(layer_sizes) - 2 else nn.Identity()
        layers += (linear, act)

    return nn.Sequential(*layers)

In [10]:
np.load(
    'C:/Users/evani/OneDrive/AI leiden/Sanquin/RL_matching-main/NN training data/reg_ABDCcEeKkFyaFybJkaJkbMNSs/Q_matrices_0_0.pickle',
    allow_pickle=True)

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [163]:
f = 'C:/Users/evani/OneDrive/AI leiden/Sanquin/RL_matching-main/NN training data/reg_ABDCcEeKkFyaFybJkaJkbMNSs/q_matrices/Q_matrices_0_0.pickle'
s = 'C:/Users/evani/OneDrive/AI leiden/Sanquin/RL_matching-main/NN training data/reg_ABDCcEeKkFyaFybJkaJkbMNSs/states/states_0_0.pickle'

In [164]:
x = torch.load(f, pickle_module=pickle)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [59]:
objects = []
with (open(f, 'rb')) as openfile:
    while True:
        try:
            objects.append((pickle.load(openfile)))
        except EOFError:
            break

In [66]:
np.load(f, allow_pickle=True)

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [70]:
np.load(s, allow_pickle=True)

array([[ 2.,  2., 14., ...,  0.,  0.,  0.],
       [ 2.,  2., 13., ...,  0.,  0.,  0.],
       [ 2.,  2., 13., ...,  0.,  0.,  0.],
       ...,
       [10., 19.,  0., ...,  0.,  0.,  0.],
       [10., 19.,  0., ...,  0.,  0.,  0.],
       [10., 19.,  0., ...,  0.,  0.,  0.]])

In [87]:
layers = [128, 128, 128]
input_size = 72
output = 8

In [ ]:
import

In [2]:
def build_nn():
    input_size = [22]
    output = [8]
    layer_sizes = input_size + [128, 128, 128] + output

    assert len(layer_sizes) > 1
    layers = []
    for index in range(len(layer_sizes) - 1):
        linear = nn.Linear(layer_sizes[index], layer_sizes[index + 1])
        act = nn.ReLU() if index < len(layer_sizes) - 2 else nn.Identity()
        layers += (linear, act)
    return nn.Sequential(*layers)

In [ ]:
model = build_nn()

NameError: name 'nn' is not defined

In [ ]:
print(model)

In [156]:
model(torch.rand(80))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x80 and 72x128)

In [95]:
print(model)

Sequential(
  (0): Linear(in_features=72, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=128, bias=True)
  (5): ReLU()
  (6): Linear(in_features=128, out_features=8, bias=True)
  (7): Identity()
)


In [96]:
[72] + [8] + [128, 128]

[72, 8, 128, 128]

In [94]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print
        name, param.data

In [98]:
pd.DataFrame([500], ['column'])

,0
column,500


In [130]:
import optuna
from optuna.trial import TrialState
import random

In [185]:
def define_model():
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = 3
    layers = []

    in_features = 72
    for i in range(n_layers):
        out_features = random.randint(72, 256)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())

        in_features = out_features
    layers.append(nn.Linear(in_features, 8))

    return nn.Sequential(*layers)

In [187]:
model = define_model()

In [188]:
print(model)

Sequential(
  (0): Linear(in_features=72, out_features=135, bias=True)
  (1): ReLU()
  (2): Linear(in_features=135, out_features=156, bias=True)
  (3): ReLU()
  (4): Linear(in_features=156, out_features=74, bias=True)
  (5): ReLU()
  (6): Linear(in_features=74, out_features=8, bias=True)
)


In [4]:
import torch.nn as nn

In [28]:
def define_model():
    layers = []

    in_features = 72
    non = [128, 128, 128]
    p = [0.34, 0.66, 0.11]
    output = 8
    for i in range(len(non) - 1):
        out_features = non[i]
        act = nn.ReLU()
        linear = nn.Linear(in_features, out_features)
        layers += (linear, act)
        layers.append(nn.Dropout(p[i]))
        in_features = out_features
    layers.append((nn.Linear(in_features, output)))

    return nn.Sequential(*layers)

In [32]:
model = define_model()

In [33]:
print(model)

Sequential(
  (0): Linear(in_features=72, out_features=128, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.34, inplace=False)
  (3): Linear(in_features=128, out_features=128, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.66, inplace=False)
  (6): Linear(in_features=128, out_features=8, bias=True)
)


In [1]:
'results/kickstart/{}/loss_1.csv'.format(1)

'results/kickstart/1/loss_1.csv'

In [2]:
import torch

torch.rand(5, 4, requires_grad=False)

tensor([[0.5296, 0.8692, 0.7453, 0.3743],
        [0.9395, 0.7282, 0.1014, 0.0806],
        [0.3397, 0.8241, 0.5919, 0.1700],
        [0.4197, 0.6870, 0.4039, 0.1415],
        [0.6582, 0.0717, 0.1942, 0.7462]])

In [23]:
import torch
from torch.nn import BCELoss
from torch.nn import CrossEntropyLoss, MSELoss

loss = CrossEntropyLoss()

pred = torch.Tensor([0.2, 0.7, 0.1])
true = torch.Tensor([1, 0, 0])

loss(true, pred)
# tensor(0.)

tensor(1.3514)

In [54]:
import pandas as pd

demand = pd.read_csv('C:/Users/evani/OneDrive/AI leiden/Sanquin/RL_matching-main/supply/147300/cau100_afr0_asi0_0.csv')

In [58]:
demand.loc[(demand['A'] == 1) & (demand['D'] == 1)]

,A,B,D,C,c,E,e,K,k,M,N,S,s,Fya,Fyb,Jka,Jkb,Ethnicity,Index
3,1,0,1,1,1,0,1,0,1,1,1,1,1,0,1,1,0,Caucasian,3
6,1,0,1,1,1,0,1,0,1,1,1,0,1,1,1,0,1,Caucasian,6
12,1,0,1,0,1,1,1,0,1,1,1,1,1,1,0,1,1,Caucasian,12
13,1,0,1,1,1,0,1,0,1,1,0,1,0,0,1,1,0,Caucasian,13
17,1,0,1,1,1,0,1,0,1,1,1,1,1,1,1,0,1,Caucasian,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147268,1,0,1,0,1,1,1,0,1,1,1,1,0,0,1,1,1,Caucasian,147268
147269,1,0,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,Caucasian,147269
147276,1,0,1,1,0,0,1,0,1,1,1,1,1,1,1,0,1,Caucasian,147276
147277,1,0,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,Caucasian,147277


In [59]:
demand

,A,B,D,C,c,E,e,K,k,M,N,S,s,Fya,Fyb,Jka,Jkb,Ethnicity,Index
0,0,0,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,Caucasian,0
1,0,1,1,1,1,0,1,0,1,1,1,0,1,0,1,0,1,Caucasian,1
2,0,0,1,1,1,0,1,0,1,1,1,1,1,0,1,1,1,Caucasian,2
3,1,0,1,1,1,0,1,0,1,1,1,1,1,0,1,1,0,Caucasian,3
4,0,1,1,1,1,0,1,0,1,1,1,0,1,0,1,1,1,Caucasian,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147280,0,0,1,1,1,1,1,0,1,0,1,0,1,1,0,1,0,Caucasian,147280
147281,0,0,1,1,0,0,1,0,1,1,1,1,0,1,1,1,0,Caucasian,147281
147282,1,0,1,1,1,0,1,0,1,0,1,1,1,0,1,1,1,Caucasian,147282
147283,0,0,0,0,1,0,1,0,1,1,0,0,1,1,1,1,0,Caucasian,147283
